# **Semiparametric SVM training using subgradients in Spark **

#### bla, bla, bla. 

#### We will benchmark the algorithms with data files from UCI:

* **Ripley**, the Ripley dataset
* **Kwok**, the Kwok dataset
* **Twonorm**, the Twonorm dataset
* **Waveform**, the Waveform dataset
* **Covertype**, the Covertype dataset


In [3]:
# definir variable de usuario y ejecutar condicionalmente cualquier código que dependa del contexto de ejecución
# no borrar código, simplemente ejecutarlo o no con un "if"
# cada uno mantiene actualizada su parte del "if" y no toca la del otro

user = 'navia'
#user = 'roberto'

#modelo = 'hybrid' 
#modelo = 'kernelgrad' 
overwrite_results = True # overwrites results even when the result file exists, skips the execution otherwise

if user == 'roberto':
    # definir sc
    import findspark
    findspark.init()
    from pyspark import SparkConf, SparkContext
    conf = (SparkConf().setMaster("local[4]").setAppName("My app").set("spark.executor.memory", "2g"))
    sc = SparkContext(conf = conf)
    import common.lib.svm_utils as SVM_UTILS
    import common.lib.quadtree_utils as QUADTREE
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    import pickle
    from math import sqrt
    %matplotlib inline
    # Se importan las funciones
    from common.lib.SGMAUtils import SGMA
    from common.lib.IRWLSUtils import IRWLS
    from common.lib.ResultsUtils import show_results
    Npartitions = 12
    Samplefraction = 0.05

if user == 'navia':
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/svm_utils.py")
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/IRWLSUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/KernelUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/ResultsUtils.py")   
    sc.addPyFile("file:///export/usuarios01/navia/spark/SVM_spark/common/lib/SGMAUtils.py")   
    import svm_utils as SVM_UTILS
    #from SGMAUtils import SGMA
    from IRWLSUtils import loadFile, train_SGMA_IRWLS, train_random_IRWLS
    #from ResultsUtils import compute_AUCs
    import numpy as np
    from pyspark.mllib.regression import LabeledPoint
    from pyspark.mllib.linalg import SparseVector, DenseVector
    import pickle
    from math import sqrt
    from pyspark.mllib.util import MLUtils
    %matplotlib inline
    Npartitions = 32
    Samplefraction = 0.05

# 0 = Ripley
# 1 = Kwok
# 2 = Twonorm
# 3 = Waveform
# 4 = Adult
# 5 = SUSY
# 6 = KddCup1999

datasets = [0, 1, 2, 3, 4, 5, 6]
folds = [0, 1, 2, 3, 4]
dataset_names = ['Ripley', 'Kwok', 'Twonorm', 'Waveform', 'Adult', 'Susy', 'KddCup1999']
Niters = [50, 100, 200]
NCs = [5, 10, 25, 50, 100, 200]
modelos = ['hybrid', 'kernelgrad', 'SGMA_IRWLS', 'LinearSVM', 'Logistic', 'random_IRWLS']

datasets = [4]
folds = [0]
modelos = ['random_IRWLS']
Niters = [150]
NCs = [150]

for modelo in modelos:
    for kdataset in datasets:
        for kfold in folds:
            for Niter in Niters:
                C = 100.0
                name_dataset = dataset_names[kdataset]
                
                #####################################################################################################################
                # DATA LOADING: the result of this part must always be three RDDs for train, val and test, containing labelled points.
                #####################################################################################################################
                if kdataset in [0, 1, 2, 3]: # loading from .mat
                    x_tr, y_tr, x_val, y_val, x_tst, y_tst = SVM_UTILS.load_data(kdataset, kfold)
                    NI = x_tr.shape[1]
                    sigma = sqrt(NI)
                    XtrRDD = sc.parallelize(np.hstack((y_tr, x_tr)), Npartitions).map(lambda x: LabeledPoint(x[0], x[1:]))
                    XvalRDD = sc.parallelize(np.hstack((y_val, x_val)), Npartitions).map(lambda x: LabeledPoint(x[0], x[1:]))
                    XtstRDD = sc.parallelize(np.hstack((y_tst, x_tst)), Npartitions).map(lambda x: LabeledPoint(x[0], x[1:]))

                if kdataset in [4]: # loading libsvm format
                    if kdataset == 4:
                        dimensions = 123                       
                    print "Loading " + name_dataset
                    XtrRDD = loadFile('./data/' + name_dataset.lower() + '_train',sc,dimensions,Npartitions)
                    XvalRDD = loadFile('./data/' + name_dataset.lower() + '_val',sc,dimensions,Npartitions)
                    XtstRDD = loadFile('./data/' + name_dataset.lower() + '_test',sc,dimensions,Npartitions)
                    sigma = np.sqrt(dimensions)
                    
                if kdataset in [5]: # loading in libsvm format and splitting RDD
                    if kdataset == 5:
                        filename = 'file:///export/usuarios01/navia/spark/SVM_spark/data/SUSY.txt'
                        #filename = 'file:///export/usuarios01/navia/spark/SVM_spark/data/minisusy.txt'
                        dimensions = 18
                    rawdata = MLUtils.loadLibSVMFile(sc, filename)
                    # Labelled points are sparse, we map the values
                    rawdata = rawdata.map(lambda x: LabeledPoint(x.label, DenseVector((x.features).toArray())  ))
                    XtrRDD, XvalRDD, XtstRDD = rawdata.randomSplit(weights=[0.7, 0.1, 0.2], seed=1234)
                    sigma = np.sqrt(dimensions)
                
                #XtrRDD.cache()
                #XvalRDD.cache()
                #XtstRDD.cache()
                #################################   END LOADING DATA ##########################################################
                
                for NC in NCs:
                    print "Dataset = %s, modelo = %s, kfold = %d, Niter = %d, NC = %d" % (name_dataset, modelo, kfold, Niter, NC)
                    filename = './results/dataset_' + str(kdataset) + '_modelo_' + modelo + '_NC_' + str(NC) + '_Niter_' + str(Niter) + '_kfold_' + str(kfold) + '.pkl'
                    #import code
                    #code.interact(local=locals())
                    try:
                        f = open(filename,'r')
                        f.close()
                        file_exists = True
                    except:
                        file_exists = False
                        pass
                    execute = False
                    if file_exists:
                        if overwrite_results:
                            execute = True
                    else:
                        execute = True                                  
                    if execute:

                        if modelo == 'hybrid':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_hybridSVM(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter, Samplefraction)

                        if modelo == 'kernelgrad':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_kernelgrad(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter, Samplefraction)

                        if modelo == 'SGMA_IRWLS':
                            auc_tr, auc_val, auc_tst, exe_time = train_SGMA_IRWLS(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter, Samplefraction)

                        if modelo == 'LinearSVM':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_linear_SVM(XtrRDD, XvalRDD, XtstRDD)
                                                
                        if modelo == 'Logistic':
                            auc_tr, auc_val, auc_tst, exe_time = SVM_UTILS.train_logistic(XtrRDD, XvalRDD, XtstRDD)

                        if modelo == 'random_IRWLS':
                            auc_tr, auc_val, auc_tst, exe_time = train_random_IRWLS(XtrRDD, XvalRDD, XtstRDD, sigma, C, NC, Niter, Samplefraction)

                        with open(filename, 'w') as f:
                            pickle.dump([auc_tr, auc_val, auc_tst, exe_time], f)


Loading Adult
Dataset = Adult, modelo = random_IRWLS, kfold = 0, Niter = 150, NC = 150
Iteration 1 : DeltaW/W inf , Iteration Time 465.718039989
Iteration 2 : DeltaW/W 1.89047430734 , Iteration Time 381.400480032
Iteration 3 : DeltaW/W 0.418001925944 , Iteration Time 324.484227896
Iteration 4 : DeltaW/W 0.22098227886 , Iteration Time 421.412127018
Iteration 5 : DeltaW/W 0.184516027983 , Iteration Time 322.016273975
Iteration 6 : DeltaW/W 0.150905432438 , Iteration Time 302.181411982
Iteration 7 : DeltaW/W 0.158643723559 , Iteration Time 160.094115973
Iteration 8 : DeltaW/W 0.130014813119 , Iteration Time 387.525722027
Iteration 9 : DeltaW/W 0.117642517445 , Iteration Time 308.408554792
Iteration 10 : DeltaW/W 0.0918459229849 , Iteration Time 201.646561146
Iteration 11 : DeltaW/W 0.119835615615 , Iteration Time 294.946087122
Iteration 12 : DeltaW/W 0.0979460316528 , Iteration Time 371.32779789
Iteration 13 : DeltaW/W 0.101046216633 , Iteration Time 324.334986925
Iteration 14 : DeltaW/W 